Wind speed forecasts for python app (ECMWF)

In [ ]:
import os
import time
from ecmwf.opendata import Client

# Pfad zu den gespeicherten Wetterdaten
data_file_path = "data_europe.grib2"
# Festgelegtes Zeitintervall für Aktualisierungen (in Sekunden)
time_threshold = 10 * 3600  # 10 Stunden in Sekunden
overwrite = 0 # force overwriting of data

# Funktion, um das Alter der Datei zu überprüfen
def is_data_stale(file_path, time_threshold):
    if not os.path.exists(file_path):
        # Datei existiert nicht, Daten müssen abgerufen werden
        return True
    # Alter der Datei ermitteln (in Sekunden seit der letzten Änderung)
    file_age = time.time() - os.path.getmtime(file_path)
    # Überprüfen, ob die Datei älter als der festgelegte Schwellenwert ist
    return file_age > time_threshold

# ECMWF-Client initialisieren
client = Client(
    source="ecmwf",
    model="ifs",
    resol="0p25"
)

# all available steps up to 7 days (168 hours)
step_selection = [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60,
     63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120,
     123, 126, 129, 132, 135, 138, 141, 144, 150, 156, 162, 168]
# all available steps
# step_selection = [
#     0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60,
#     63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120,
#     123, 126, 129, 132, 135, 138, 141, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204,
#     210, 216, 222, 228, 234, 240
# ]

if is_data_stale(data_file_path, time_threshold) or overwrite:
    print("Daten sind veraltet, existieren nicht oder sollen überschrieben werden. Abruf neuer Daten.")
    # Abrufen der API-Daten, da sie entweder fehlen oder älter als 10 Stunden sind
    result = client.retrieve( # retrieve data worldwide, because no area tag available (https://github.com/ecmwf/ecmwf-opendata, https://www.ecmwf.int/en/forecasts/datasets/open-data)
        type="fc",  
        param=["100v", "100u"],  # U- und V-Komponenten der Windgeschwindigkeit
        target=data_file_path,
        time=0,  # Vorhersagezeit (Modelllauf um 00z)
        step=step_selection
    )
    print("Neue Daten wurden erfolgreich abgerufen und gespeichert.")
else:
    print("Daten sind aktuell und werden verwendet.")


countries (Shapefile)<br>
regions (CSV)<br>
places (CSV)

In [ ]:
import geopandas as gpd
import glob
import pandas as pd
import folium




# countries

# Pfad zu den Shapefiles (z.B. alle .shp-Dateien in einem Verzeichnis)
shapefile_path = "C:/Users/alexa/Documents/Webapp/GPC-BNDR-ADM-HPR-SAMPLE-WO-SELECTED_SHP/*.shp"

# Lade alle Shapefiles und kombiniere sie in ein GeoDataFrame
gdf_list = []
for shp in glob.glob(shapefile_path):
    gdf = gpd.read_file(shp)
    gdf_list.append(gdf)

# Kombiniere alle GeoDataFrames
all_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

#folium.GeoJson(all_gdf).add_to(map)




regions = pd.read_csv("C:/Users/alexa/Documents/Webapp/GPC-POST-GEO-SAMPLE-WO-SELECTED/CSV/GPC-REGIONS-GEO-SAMPLE-WO-SELECTED.csv", sep=';')

for index, row in regions.iterrows():
    # Erstelle eine Liste von Koordinaten aus den Spalten 'longitude' und 'latitude'
    coordinates = [(row['latitude'], row['longitude'])]  # Hier wird die Liste mit einem Tuple von Longitude und Latitude erstellt

    # Erstelle ein Polygon
    folium.Polygon(
        locations=coordinates,
        color='blue',  # Farbe des Polygons
        fill=True,
        fill_color='blue',  # Füllfarbe
        fill_opacity=0.6,
        popup=row['region1']  # Name oder weitere Informationen zur Region
    ).add_to(map)



places = pd.read_csv("C:/Users/alexa/Documents/Webapp/GPC-POST-GEO-SAMPLE-WO-SELECTED/CSV/GPC-PLACES-GEO-SAMPLE-WO-SELECTED.csv", sep=';')

for index, row in places.iterrows():
    # Erstelle eine Liste von Koordinaten aus den Spalten 'longitude' und 'latitude'
    coordinates = [(row['latitude'], row['longitude'])]  # Hier wird die Liste mit einem Tuple von Longitude und Latitude erstellt

    # Erstelle ein Polygon
    folium.Polygon(
        locations=coordinates,
        color='blue',  # Farbe des Polygons
        fill=True,
        fill_color='blue',  # Füllfarbe
        fill_opacity=0.6,
        popup=row['postcode']  # Name oder weitere Informationen zur Region
    ).add_to(map)




#Creates a map with Mapbox; without tiles and API_key specifiers, a map with OpenStreetMap is created
map = folium.Map(
    location=[52.52, 13.40],
    zoom_start=6,
    tiles=f'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/{{z}}/{{x}}/{{y}}?access_token=pk.eyJ1IjoiZWwtZ2lmIiwiYSI6ImNtMXQyYWdsYzAwMGUycXFzdmY2eDFnaWMifQ.yirQoMK5TCdmZZUFUNXxwA',
    attr='Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, <a href="https://www.mapbox.com/">Mapbox</a>'
)

folium.GeoJson(r"C:\Users\alexa\Documents\Webapp\geonames-postal-code@public.geojson").add_to(map)

for i, lat in enumerate(ds['latitude'].values[::10]):
    for j, lon in enumerate(ds['longitude'].values[::10]):
        speed = wind_speed[i, j]  # Windgeschwindigkeit an der Position (i, j)
        folium.Marker(
            [lat, lon], 
            popup=f"Windgeschwindigkeit: {speed:.2f} m/s"
        ).add_to(map)

map.save("map.html")

postal codes (OpenDataSoft)

In [ ]:
import requests
import geopandas as gpd
import folium

# API URL and parameters
base_api_url = "https://data.opendatasoft.com/api/explore/v2.1/catalog/datasets/geonames-postal-code@public/records"
limit = 20 # API pagination
total_count = 999999  # Total number of records expected
all_geojson_data = []  # List to store all geojson data

# Fetch all pages of data
for offset in range(0, total_count, limit):
    api_url = f"{base_api_url}?limit={limit}&offset={offset}"
    
    # Send API request
    response = requests.get(api_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the data in JSON format
        data = response.json()
        
        # Check if 'results' exists in the data
        if 'results' in data:
            # Extract GeoJSON data from JSON
            geojson_data = data['results']
            all_geojson_data.extend(geojson_data)  # Add the current results
        else:
            print("Key 'results' not found in the data.")
            break  # End loop if no results are found
    else:
        print(f"Request failed with status code: {response.status_code}, offset: {offset}")
        break  # End loop if an error occurs

# Convert GeoJSON data to a GeoDataFrame
features = [
    feature['geo_shape'] for feature in all_geojson_data if 'geo_shape' in feature
]

# Create the GeoDataFrame and set the CRS
gdf2 = gpd.GeoDataFrame.from_features(features)
gdf2.set_geometry('coordinates', inplace=True)  # Set 'coordinates' column as geometry
gdf2.set_crs(epsg=4326, inplace=True)  # Set CRS to EPSG:4326





# Erstelle eine Folium-Karte
m = folium.Map(location=[20, 0], zoom_start=2)  # Setze den Mittelpunkt und den Zoom der Karte

# Füge die GeoDataFrame-Daten zur Karte hinzu
GeoJson(gdf2).add_to(m)


# Speichere die Karte als HTML-Datei oder zeige sie an
m.save('world_boundaries_map.html')


Country Boundaries (OpenDataSoft)<br>
WPP (HeatMap and MarkerCluster) (OpenStreetMap)

In [ ]:
import requests
import geopandas as gpd
import folium
from folium import GeoJson
from folium.plugins import HeatMap, MarkerCluster



# API URL und Parameter für die Ländergrenzen
base_api_url = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/world-administrative-boundaries/records"
limit = 20  # API-Paginierung
total_count = 256  # Gesamtzahl der erwarteten Datensätze
all_geojson_data = []  # Liste zur Speicherung aller GeoJSON-Daten

# Fetch all pages of data
for offset in range(0, total_count, limit):
    api_url = f"{base_api_url}?limit={limit}&offset={offset}"
    
    # API Anfrage senden
    response = requests.get(api_url)

    # Überprüfen, ob die Anfrage erfolgreich war
    if response.status_code == 200:
        # Die Daten im JSON-Format extrahieren
        boundary_data = response.json()
        
        # Überprüfen, ob 'results' in den Daten vorhanden ist
        if 'results' in boundary_data:
            # GeoJSON-Daten aus dem JSON extrahieren
            geojson_data = boundary_data['results']
            all_geojson_data.extend(geojson_data)  # Füge die aktuellen Ergebnisse hinzu
        else:
            print("Key 'results' nicht in den Daten gefunden.")
            break  # Beende die Schleife, wenn keine Ergebnisse gefunden werden
    else:
        print(f"Fehler bei der Anfrage: {response.status_code}")
        break  # Beende die Schleife, wenn ein Fehler auftritt

# Konvertieren der GeoJSON-Daten in ein GeoDataFrame
features = []
for feature in all_geojson_data:
    if 'geo_shape' in feature:
        geom = feature['geo_shape']['geometry']  # Behalte die Geometrie
        properties = {
            'name': feature.get('name', None)  # Füge den Namen hinzu
        }
        features.append({
            'geometry': geom,
            'properties': properties
        })

# Erstelle das GeoDataFrame und setze das CRS
gdf1 = gpd.GeoDataFrame.from_features(features)
gdf1.set_crs(epsg=4326, inplace=True)  # Setze das CRS auf EPSG:4326




# Windkraftanlagen-Daten abrufen
url = "https://overpass-api.de/api/interpreter?data=[out:json][timeout:99999];node['power'='generator']['generator:source'='wind'](49,9,50,10);out body;" # Europe: (34.8,-30,71.6,60)
response = requests.get(url)

if response.status_code == 200:
    wind_data = response.json()
else:
    print("Fehler beim Abrufen der Windkraftdaten:", response.status_code)
    wind_data = None

# Karte erstellen
m = folium.Map(location=[50.68, 6.72], zoom_start=12)  # Erftstadt-Mitte

# Markercluster-Layer erstellen
marker_layer = folium.FeatureGroup(name="Marker Cluster")
marker_cluster = MarkerCluster().add_to(marker_layer)

# Heatmap-Layer erstellen
heatmap_layer = folium.FeatureGroup(name="Heatmap")

# Kapazitäten nach Land summieren
if wind_data:
    # Initialisiere ein Dictionary zur Speicherung der Kapazitäten nach Ländern
    capacities_by_country = {name: 0 for name in gdf1['name']}  # Verwende die Namen aus gdf1
    heat_data = []
    
    for element in wind_data["elements"]:
        try:
            # Extrahiere die Kapazität aus den tags
            if 'generator:output:electricity' in element['tags']:
                capacity_value = element['tags']['generator:output:electricity']
                # Prüfen und umrechnen je nach Einheit
                if 'kW' in capacity_value:
                    capacity = float(capacity_value.replace('kW', '').strip())
                elif 'MW' in capacity_value:
                    capacity = float(capacity_value.replace('MW', '').strip()) * 1000  # Umrechnung in kW
                elif capacity_value.lower() == 'yes':
                    capacity = 0  # Wenn keine Kapazität angegeben ist
                else:
                    capacity = float(capacity_value)  # Konvertiere direkt zu float, falls keine Einheit
            else:
                capacity = 0  # Wenn keine Kapazität angegeben ist

        except ValueError:
            capacity = 0  # Fehler beim Konvertieren

        # Überprüfen, in welchem Land sich die Windkraftanlage befindet
        point = gpd.points_from_xy([element['lon']], [element['lat']])
        for idx, country in gdf1.iterrows():
            if country['geometry'].contains(point[0]):
                capacities_by_country[country['name']] += capacity

        # Daten zur Heatmap hinzufügen
        heat_data.append([element["lat"], element["lon"], capacity])

        # Marker mit Tooltip zur Markergruppe hinzufügen
        folium.Marker(
            location=[element['lat'], element['lon']],
            tooltip=(
                f'Kapazität: {capacity} kW\n'
                f'Baujahr: {element["tags"].get("construction", "n/a")}\n'
                f'Betreiber: {element["tags"].get("operator", "n/a")}\n'
                f'Hub height: {element["tags"].get("height:hub", "n/a")} m\n'
                f'Rotor diameter: {element["tags"].get("rotor:diameter", "n/a")} m\n'
            )
        ).add_to(marker_cluster)

    # Heatmap zum Heatmap-Layer hinzufügen
    HeatMap(data=heat_data, radius=15, max_zoom=13).add_to(heatmap_layer)

    # Heatmap und Marker-Layer zur Karte hinzufügen
    heatmap_layer.add_to(m)
    marker_layer.add_to(m)

    # Füge die Ländergrenzen zur Karte hinzu
    GeoJson(gdf1.geometry).add_to(m)

    folium.Choropleth(
        geo_data=gdf1,
        name='choropleth',
        data=capacities_by_country,  # Dein Dictionary mit den Kapazitäten
        columns=['name', 'capacity'],  # Hier muss "capacity" vorhanden sein
        key_on='feature.properties.name',  # Schlüssel zur Zuordnung der Länder
        fill_color='YlGn',  # Farbskala
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Windkraftkapazität (kW)'
    ).add_to(m)

    # Füge Layer Controls hinzu
    folium.LayerControl().add_to(m)

    # Karte speichern (optional)
    m.save("wind_energy.html") # just for own visualisation, not used by app.py

    # Überprüfen, ob Features vorhanden sind
    print(f"{len(wind_data['elements'])} Windkraftdaten erfolgreich zur Karte hinzugefügt")
else:
    print("Keine Windkraftdaten verfügbar.")


In [6]:
import requests
import json
from ipyleaflet import Map, Marker, MarkerCluster
from ipywidgets import Layout

# Define the URL for Overpass API query
url = "https://overpass-api.de/api/interpreter?data=[out:json][timeout:99999];node['power'='generator']['generator:source'='solar'](49,9,50,10);out body;"

# Fetch data from Overpass API
response = requests.get(url)
data = response.json()

# Extract relevant data: latitude, longitude, and name (if available)
locations = []
for element in data['elements']:
    lat = element.get('lat')
    lon = element.get('lon')
    name = element.get('tags', {}).get('name', 'PV Generator')
    locations.append((lat, lon, name))

# Initialize the map centered on the area of interest
m = Map(center=(49.5, 9.5), zoom=10, layout=Layout(width='100%', height='600px'))

# Create markers and add them to a MarkerCluster
markers = [Marker(location=(lat, lon), title=name) for lat, lon, name in locations]
marker_cluster = MarkerCluster(markers=markers)

# Add the MarkerCluster to the map
m.add_layer(marker_cluster)

# Display the map
m


Map(center=[49.5, 9.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [ ]:
import requests
import pandas as pd

# Windkraftanlagen-Daten abrufen
url = "https://overpass-api.de/api/interpreter?data=[out:json][timeout:180];node['power'='generator']['generator:source'='solar'](40,9,50,10);out body;"
response = requests.get(url)

# Überprüfen, ob die Anfrage erfolgreich war und die Daten enthalten
if response.status_code == 200:
    wind_data = response.json().get('elements', [])
    
    # DataFrame erstellen
    df = pd.DataFrame(wind_data)

    # Tags in eigene Spalten erweitern
    tags_df = pd.json_normalize(df['tags'])
    
    # Die Tags-Spalten mit dem ursprünglichen DataFrame zusammenführen und einen Suffix für überlappende Spalten verwenden
    df = df.drop(columns=['tags']).join(tags_df, rsuffix='_tag')

    # Die ersten Zeilen anzeigen
        # Alle Spalten anzeigen
    pd.set_option('display.max_columns', None)

    # Die ersten Zeilen des DataFrames anzeigen
    display(df.head())

    df.to_excel("Solaranlagen_Daten.xlsx", index=False)
    print("Die Tabelle wurde erfolgreich als 'Solaranlagen_Daten.xlsx' gespeichert.")
else:
    print("Fehler beim Abrufen der Solarkraftdaten:", response.status_code)


,type,id,lat,lon,description,generator:method,generator:output:electricity,generator:source,generator:type,power,location,note,generator:output:hot_water,name,generator:place,operator,source,generator:location,survey:date,building,layer,generator:output:electricity:energy,diameter,source:width,generator:orientation,wheelchair,amenity,bicycle,fee,motorcar,socket:schuko,start_date,video,generator:output:heat,addr:street,opening_hours,website,frequency,voltage,generator:plant,denotation,leaf_type,natural,operator:wikidata,fixme,generator:solar:modules,bench,lit,lockable,socket:device:USB-A,socket:device:qi-wireless,colour,direction,generator:solar:tracking
0,node,701629811,48.663324,9.203941,Solarzellen auf dem Dach der Lagerhalle,photovoltaic,yes,solar,solar_photovoltaic_panel,generator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,994865515,49.649236,9.272201,Mover,photovoltaic,NaN,solar,solar_photovoltaic_panel,generator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,1041095988,49.649142,9.273090,Mover,photovoltaic,NaN,solar,solar_photovoltaic_panel,generator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,1041095995,49.649298,9.271608,Mover,photovoltaic,NaN,solar,solar_photovoltaic_panel,generator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,1041096004,49.649329,9.271312,Mover,photovoltaic,NaN,solar,solar_photovoltaic_panel,generator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Die Tabelle wurde erfolgreich als 'Windkraftanlagen_Daten.xlsx' gespeichert.


Country Boundaries (OpenStreetMap)<br>
WPP (HeatMap and MarkerCluster) (OpenStreetMap)


In [ ]:
import requests
import geopandas as gpd
from ipyleaflet import Map, Marker, MarkerCluster, GeoJSON, Heatmap

# Funktion zum Abrufen von Windkraftanlagen
def fetch_wind_data():
    wind_url = "https://overpass-api.de/api/interpreter?data=[out:json][timeout:99999];node['power'='generator']['generator:source'='wind'](49,9,50,10);out body;"
    response = requests.get(wind_url)

    if response.status_code == 200:
        wind_data = response.json()
        return wind_data['elements']
    else:
        print(f"Fehler beim Abrufen der Winddaten: {response.status_code}")
        return []

# Funktion zum Abrufen von Ländergrenzen
def fetch_country_data():
    country_url = "https://overpass-api.de/api/interpreter?data=[out:json];relation['boundary'='administrative']['admin_level'='2'];out body;"
    response = requests.get(country_url)

    if response.status_code == 200:
        country_data = response.json()
        return country_data['elements']
    else:
        print(f"Fehler beim Abrufen der Länderdaten: {response.status_code}")
        return []

# Abrufen der Daten
wind_data = fetch_wind_data()
country_data = fetch_country_data()

# Überprüfen, ob die Länderdaten abgerufen wurden
print(f"Anzahl der Länder: {len(country_data)}")

# Erstelle ein GeoDataFrame für die Länder
country_features = []
print(country_data)
for element in country_data:
    for member in element['members']:
        way_id = member['ref']
        way_url = f"https://overpass-api.de/api/interpreter?data=[out:json];way({way_id});out geom;"
        way_response = requests.get(way_url)
        if way_response.status_code == 200:
            way_data = way_response.json()
            for way in way_data['elements']:
                name = element['tags'].get('name', 'Unbekannt')
                country_features.append({'geometry': way['geometry'], 'properties': name})
                print(f'{name} appended')

# Erstelle ein GeoDataFrame und setze die Geometrie manuell
gdf_countries = gpd.GeoDataFrame(country_features)
gdf_countries.set_geometry('geometry', inplace=True)  # Setze die Geometriespalte
gdf_countries.set_crs(epsg=4326, inplace=True)  # Setze das CRS auf EPSG:4326

# Erstelle die ipyleaflet-Karte
m = Map(center=(50, 9.5), zoom=6)

# Füge die Ländergrenzen hinzu, wenn gdf_countries nicht leer ist
if not gdf_countries.empty:
    geojson_countries = GeoJSON(data=gdf_countries.__geo_interface__, style={'color': 'blue', 'weight': 2})
    m.add_layer(geojson_countries)

# MarkerCluster für Windkraftanlagen
marker_cluster = MarkerCluster(name="Windkraftanlagen").add_to(m)
heatmap_data = []  # Daten für die Heatmap

# Füge Windkraftanlagen als Marker hinzu und bereite die Heatmap-Daten vor
for element in wind_data:
    if 'lat' in element and 'lon' in element:
        lat = element['lat']
        lon = element['lon']
        capacity = element['tags'].get('generator:output:electricity', 'N/A')
        
        # Marker hinzufügen
        marker = Marker(location=(lat, lon), title=f'Kapazität: {capacity}')
        marker_cluster.add_child(marker)
        
        # Heatmap-Daten hinzufügen (Kapazität als Gewicht)
        heatmap_data.append([lat, lon, float(capacity.replace('kW', '').strip()) if 'kW' in capacity else 0])

# Füge die Heatmap hinzu
if heatmap_data:
    heatmap_layer = Heatmap(heatmap_data, radius=15, max_zoom=13)
    m.add_layer(heatmap_layer)

# Zeige die Karte an
m


Velocity works with viridis colormap<br>
ColormapControl works only outside Shiny environment (?)

In [4]:
# Notwendige Bibliotheken importieren
from ipyleaflet import Map, ColormapControl
from ipyleaflet.velocity import Velocity
from branca.colormap import linear
import numpy as np
from ipywidgets import Layout

# Beispielhafte Windgeschwindigkeitsdaten generieren
lats = np.linspace(35, 60, 10)  # Breitengrade
lons = np.linspace(-10, 30, 10)  # Längengrade
u = np.random.uniform(-10, 10, (len(lats), len(lons)))  # Zonalgeschwindigkeit
v = np.random.uniform(-10, 10, (len(lats), len(lons)))  # Meridionalgeschwindigkeit

# Berechne die Windgeschwindigkeit (Betrag der Geschwindigkeit)
wind_speed = np.sqrt(u**2 + v**2)

# Erstelle die Velocity Layer
velocity_layer = Velocity(
    data={
        'u': (['lat', 'lon'], u),
        'v': (['lat', 'lon'], v),
        'lat': lats,
        'lon': lons,
    },
    zonal_speed='u',
    meridional_speed='v',
    latitude_dimension='lat',
    longitude_dimension='lon',
    velocity_scale=0.01,
    max_velocity=20,
    display_options={
        'velocityType': 'Wind Speed',
        'displayPosition': 'bottomleft',
        'displayEmptyString': 'No wind data'
    }
)

# Karte erstellen
m = Map(center=[50, 10], zoom=4, layout=Layout(width='100%', height='500px'))

# Velocity Layer zur Karte hinzufügen
m.add_layer(velocity_layer)

# Colormap für Windgeschwindigkeiten basierend auf dem Betrag der Windgeschwindigkeit erstellen
min_wind_speed = wind_speed.min()  # minimale Windgeschwindigkeit
max_wind_speed = wind_speed.max()  # maximale Windgeschwindigkeit

# Eine Farbskala basierend auf dem Betrag der Windgeschwindigkeit
colormap = linear.viridis.scale(min_wind_speed, max_wind_speed)

# ColormapControl zur Karte hinzufügen
colormap_control = ColormapControl(
    caption='Windgeschwindigkeit (m/s)',
    colormap=colormap,
    value_min=min_wind_speed,
    value_max=max_wind_speed,
    position='bottomright'
)

m.add_control(colormap_control)

# Karte anzeigen
m


Map(center=[50, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [3]:
import requests

url = "https://overpass-api.de/api/interpreter?data=[out:json][timeout:99999];node['power'='generator']['generator:source'='wind'](49,9,50,10);out body;"

# Anfrage an die Overpass API senden
response = requests.get(url)
data = response.json()

# Zähler und Summen für Windkraftanlagen mit und ohne Modell
count_with_model = 0
count_without_model = 0
total_capacity_with_model = 0.0
total_capacity_without_model = 0.0

# Funktion zur Umrechnung der Kapazität in MW
def convert_to_mw(value):
    if " MW" in value:
        return float(value.replace(" MW", ""))
    elif " GW" in value:
        return float(value.replace(" GW", "")) * 1000
    elif " kW" in value:
        return float(value.replace(" kW", "")) / 1000
    elif " W" in value:
        return float(value.replace(" W", "")) / 1_000_000
    elif value.isdigit():  # Falls keine Einheit angegeben ist
        return float(value) / 1_000_000
    else:
        return None  # Falls die Umrechnung fehlschlägt

# Modelle und Kapazitäten der Windkraftanlagen ausgeben, wenn vorhanden
for element in data['elements']:  # Die ersten 20 Elemente auswählen
    tags = element['tags']
    model = tags.get('model')
    capacity_raw = tags.get('generator:output:electricity')

    # Kapazität umwandeln in MW, wenn vorhanden und umrechenbar
    capacity = convert_to_mw(capacity_raw) if capacity_raw else None
    
    # Windkraftanlagen mit Modell
    if model:
        print(f"Windkraftanlage Modell: {model}, Kapazität: {capacity} MW\n")
        count_with_model += 1
        if capacity:
            total_capacity_with_model += capacity
    # Windkraftanlagen ohne Modell
    else:
        count_without_model += 1
        if capacity:
            total_capacity_without_model += capacity

# Durchschnittskapazitäten berechnen
avg_capacity_with_model = total_capacity_with_model / count_with_model if count_with_model > 0 else 0
avg_capacity_without_model = total_capacity_without_model / count_without_model if count_without_model > 0 else 0

# Gesamtzahl und Durchschnittskapazität ausgeben
print(f"Anzahl der Windkraftanlagen mit Modell: {count_with_model}")
print(f"Durchschnittliche Kapazität der Windkraftanlagen mit Modell: {avg_capacity_with_model:.2f} MW")
print(f"Anzahl der Windkraftanlagen ohne Modell: {count_without_model}")
print(f"Durchschnittliche Kapazität der Windkraftanlagen ohne Modell: {avg_capacity_without_model:.2f} MW")


Windkraftanlage Modell: N82/1500, Kapazität: 1.5 MW

Windkraftanlage Modell: V80, Kapazität: 2.0 MW

Windkraftanlage Modell: V80, Kapazität: 2.0 MW

Windkraftanlage Modell: V80, Kapazität: 2.0 MW

Windkraftanlage Modell: V80, Kapazität: 2.0 MW

Windkraftanlage Modell: V80, Kapazität: 2.0 MW

Windkraftanlage Modell: V80, Kapazität: 2.0 MW

Windkraftanlage Modell: V80, Kapazität: 2.0 MW

Windkraftanlage Modell: V80, Kapazität: 2.0 MW

Windkraftanlage Modell: V90/2.0MW, Kapazität: 2.0 MW

Windkraftanlage Modell: V90/2.0MW, Kapazität: 2.0 MW

Windkraftanlage Modell: V80, Kapazität: 2.0 MW

Windkraftanlage Modell: NM82, Kapazität: 1.5 MW

Windkraftanlage Modell: V112/3.0MW, Kapazität: 3.0 MW

Windkraftanlage Modell: MD77, Kapazität: 1.5 MW

Windkraftanlage Modell: MD77, Kapazität: 1.5 MW

Windkraftanlage Modell: MD77, Kapazität: 1.5 MW

Windkraftanlage Modell: MD77, Kapazität: 1.5 MW

Windkraftanlage Modell: MD77, Kapazität: 1.5 MW

Windkraftanlage Modell: Nordex SE N117, Kapazität: 2.4 MW


In [ ]:
# Importieren der erforderlichen Bibliotheken
import pandas as pd

# Pfad zur Datei
file_path = r"time_series\2015_01_ActualGenerationOutputPerGenerationUnit_16.1.A_r2.1.csv"

# Laden der CSV-Datei (tab-separiert)
df = pd.read_csv(file_path, sep='\t')

# Filtern der Zeilen, die 'GenerationUnitType' entweder 'Wind Onshore' oder 'Wind Offshore' enthalten
df_filtered = df[(df['GenerationUnitType'] == 'Wind Onshore ') | (df['GenerationUnitType'] == 'Wind Offshore ')]

# Auswählen der gewünschten Spalten
df_selected = df_filtered[['DateTime (UTC)', 'GenerationUnitCode', 'ActualGenerationOutput(MW)', 'GenerationUnitInstalledCapacity(MW)']]

# Speichern des gefilterten DataFrames in einer Excel-Datei
output_file_path = r"time_series\Filtered_Wind_Data_2015_01.xlsx"
df_selected.to_excel(output_file_path, index=False)

print("Die gefilterten Daten wurden erfolgreich in", output_file_path, "gespeichert.")


In [14]:
import io
import os
os.environ['FOR_DISABLE_CONSOLE_CTRL_HANDLER'] = 'T' # to disable Ctrl+C crashing python when having scipy.interpolate imported (disables Fortrun runtime library from intercepting Ctrl+C signal and lets it to the Python interpreter)
from openpyxl import Workbook
from shiny import ui, App, reactive, render
from shinywidgets import render_widget, output_widget
from ipyleaflet import Map, Marker, MarkerCluster, WidgetControl, FullScreenControl, AwesomeIcon, Heatmap
from ipywidgets import SelectionSlider, Play, VBox, jslink, Layout, HTML  # pip install ipywidgets==7.6.5, because version 8 has an issue with popups (https://stackoverflow.com/questions/75434737/shiny-for-python-using-add-layer-for-popus-from-ipyleaflet)
import numpy as np
import pandas as pd
import xarray as xr
from scipy.interpolate import interp2d  # pip install scipy==1.13.1, interp2d much faster than RegularGridInterpolator, even if deprecated
import base64
from ecmwf.opendata import Client
from branca.colormap import linear
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import time
starttime = time.time()

# Define initial variables
initial = 0
lat_min, lat_max = 35, 72
lon_min, lon_max = -25, 45

# Path to the stored weather data
wind_file = "data/weather_forecast/data_europe.grib2"
# Set time interval for updates (in seconds)
time_threshold = 24 * 3600  # 24 hours in seconds
overwrite = 0  # force overwriting of data

# Initialise ECMWF client
client = Client(
    source="ecmwf",
    model="ifs",
    resol="0p25"
)

if os.getenv("RENDER") or os.getenv("WEBSITE_HOSTNAME"):  # for Render or Azure Server
    # for saving storage on server, only 4 time steps
    step_selection = [0, 3, 6, 9]
else:
    # all available steps up to 7 days (168 hours)
    step_selection = [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60,
                      63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120,
                      123, 126, 129, 132, 135, 138, 141, 144, 150, 156, 162, 168]

# all available steps
# step_selection = [
#     0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60,
#     63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120,
#     123, 126, 129, 132, 135, 138, 141, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204,
#     210, 216, 222, 228, 234, 240
# ]

# Function to check the age of the file, to avoid a cron-job, which is a paid feature on most Cloud Computing platforms
def is_data_stale(wind_file, time_threshold):
    if not os.path.exists(wind_file):
        # File does not exist, data must be fetched
        return True
    # Determine the age of the file (in seconds since the last modification)
    file_age = time.time() - os.path.getmtime(wind_file)
    # Check if the file is older than the specified threshold
    return file_age > time_threshold

if is_data_stale(wind_file, time_threshold) or overwrite:
    print("Data is outdated, does not exist, or should be overwritten. Fetching new data.")
    # Fetching API data since they are either missing or older than 10 hours
    result = client.retrieve(  # retrieve data worldwide, because no area tag available (https://github.com/ecmwf/ecmwf-opendata, https://www.ecmwf.int/en/forecasts/datasets/open-data)
        type="fc",
        param=["100v", "100u"],  # U- and V-components of wind speed
        target=wind_file,
        time=0,  # Forecast time (model run at 00z)
        step=step_selection
    )
    print("New data has been successfully fetched and saved.")
else:
    print("Data is current and will be used.")

# Load the wind data (Grib2 file)
ds = xr.open_dataset(wind_file, engine='cfgrib')

# Filter the data for Europe and extract relevant columns
ds_filtered = ds.sel(latitude=slice(lat_max, lat_min), longitude=slice(lon_min, lon_max))
lats = ds_filtered['latitude'].values
lons = ds_filtered['longitude'].values
u = ds_filtered['u100'].values
v = ds_filtered['v100'].values
valid_times = ds_filtered['valid_time'].values

# Filter the data for Europe and extract relevant columns
df = pd.read_parquet("data/WPPs/The_Wind_Power.parquet") # 0.7 seconds when WPPs already regionally filtered and stored as parquet file. As unfiltered excel file it takes 11 seconds
df = df.iloc[::100] # only every 100th to alleviate computational and storage burden
ids = df['ID'].values
project_names = df['Name'].values
lats_plants = df['Latitude'].values
lons_plants = df['Longitude'].values
manufacturers = df['Manufacturer']
turbine_types = df['Turbine']
hub_heights = df['Hub height']
numbers_of_turbines = df['Number of turbines']
capacities = df['Total power'].values
developers = df['Developer'].values
operators = df['Operator'].values
owners = df['Owner'].values
commissioning_dates = df['Commissioning date'].values
ages_months = df['Ages months'].values

number_wpps = len(ids)

from sklearn.preprocessing import OneHotEncoder

# Lade die gespeicherte Reihenfolge der Turbinentypen
known_turbine_types = np.load("turbine_types_order.npy")
processed_turbine_types = [turbine if turbine in known_turbine_types else "nan" for turbine in turbine_types]
encoder = OneHotEncoder(categories=[known_turbine_types], sparse_output=False)
turbine_types_onehot = encoder.fit_transform(np.array(processed_turbine_types).reshape(-1, 1))

hub_height_min = 0.9 * df['Hub height'].min()
hub_height_max = 1.1 * df['Hub height'].max()
commissioning_years = df['Commissioning date'].str.split('/').str[0].astype(int)
min_year = commissioning_years.min()
max_year = commissioning_years.max()
min_capacity = capacities.min()
max_capacity = capacities.max()

import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_size, use_dropout=False, dropout_rate=0.3, 
                 use_batch_norm=False, activation_fn=nn.ReLU):
        super(MLP, self).__init__()

        layers = []

        # Erste Schicht
        layers.append(nn.Linear(input_size, 256))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(256))
        layers.append(activation_fn())

        # Zweite Schicht
        layers.append(nn.Linear(256, 128))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(128))
        layers.append(activation_fn())

        # Dritte Schicht
        layers.append(nn.Linear(128, 64))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(64))
        layers.append(activation_fn())

        # Vierte Schicht
        layers.append(nn.Linear(64, 32))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(32))
        layers.append(activation_fn())

        # Dropout nach der letzten versteckten Schicht (optional)
        if use_dropout:
            layers.append(nn.Dropout(dropout_rate))

        # Ausgabeschicht
        layers.append(nn.Linear(32, 1))

        # Modell zusammenstellen
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)
    
import torch

model_path = "trained_model_with_metadata.pth"

# Lade die Metadaten und den Modellzustand
loaded_data = torch.load(model_path)

# Extrahiere die Metadaten
input_size = loaded_data["input_size"]
state_dict = loaded_data["model_state_dict"]

# Initialisiere das Modell mit der geladenen `input_size`
model = MLP(input_size)
model.load_state_dict(state_dict)
model.eval()

print(f"Modell erfolgreich geladen mit input_size={input_size}")

# Interpolation period and interval
start_time = valid_times[0]
end_time = valid_times[-1]
total_hours = int((end_time - start_time) / np.timedelta64(1, 'h'))
step_size = np.timedelta64(1, 'h')
step_size_hours = step_size / np.timedelta64(1, 'h')
valid_times_interpol = [start_time + i * step_size for i in range(int(total_hours / step_size_hours))]

# Calculate total wind speed and convert to 3D array
total_selection = np.array([np.sqrt(u_value**2 + v_value**2) for u_value, v_value in zip(u, v)])

# Function for temporal interpolation
def interpolation(list):
    list_interpol = []
    for i in range(len(list) - 1):
        step_start = valid_times[i]
        step_end = valid_times[i + 1]
        list_start = list[i]
        list_end = list[i + 1]
        list_interpol.append(list_start)
        interval = step_end - step_start
        for step in np.arange(step_start + step_size, step_end, step_size):
            factor = (step - step_start) / interval
            interpolated_value = (1 - factor) * list_start + factor * list_end
            list_interpol.append(interpolated_value)
    list_interpol.append(list[-1])
    return list_interpol

# Interpolation
total_selection_interpol = interpolation(total_selection)

valid_times_interpol = []
steps = int(total_hours / step_size_hours)
for i in range(steps):
    valid_times_interpol.append(start_time + i * step_size)

example_data = pd.read_parquet("data/production_history/Example/example_time_series.parquet")
example_dates = example_data['Date']
example_production = example_data['Production (kW)']

app_ui = ui.page_navbar(
    ui.nav_panel(
        "WPP database",
        output_widget("map")
    ),
    ui.nav_panel(
        "Customise WPP",
        ui.row(
            ui.column(2,  # Left column for input fields
                ui.input_numeric("lat", "Turbine Latitude", min=lat_min, max=lat_max, value=(lat_min + lat_max) / 2),
                ui.input_numeric("lon", "Turbine Longitude", min=lon_min, max=lon_max, value=(lon_min + lon_max) / 2),
                ui.input_select("turbine_type", "Turbine Type", choices=known_turbine_types.tolist(), selected=known_turbine_types[0]),
                ui.input_slider("hub_height", "Turbine Hub Height (m)", min=hub_height_min, max=hub_height_max, value=(hub_height_min + hub_height_max) / 2),
                ui.input_slider("commissioning_date_year", "Commissioning Date (Year)", min=min_year, max=max_year, value=(min_year + max_year) / 2, step=1),
                ui.input_slider("commissioning_date_month", "Commissioning Date (Month)", min=1, max=12, value=6, step=1),
                ui.input_slider("capacity", "Capacity (kW)", min=min_capacity, max=max_capacity, value=(min_capacity + max_capacity) / 2),
                ui.tags.br(),
                ui.input_file("upload_file", "Contribute data for this configuration", accept=[".xlsx"]),
                ui.input_action_link("download_example", "Download Example File")
            ),
            ui.column(10,  # Right column for output
                ui.panel_well(  # Panel to centre the content
                    ui.output_ui("output_summary"),
                    ui.tags.br(),
                    ui.output_plot("output_graph"),
                    ui.tags.br(),
                    ui.input_action_button("action_button", "Download Forecast")
                ),
            )
        ),
        value='customise_WPP'
    ),
    ui.nav_panel(
        "Documentation",
        ui.h3("Wind Power Forecast Application"),
        ui.p(
            "This web application visualises wind power production forecasts for wind power plants across Europe. "
            "Users can view wind plant information, forecasted wind speeds, and production values on an interactive map. "
            "Each wind plant can be customised by selecting its attributes, and predictions are displayed based on input parameters."
            "All given times are in Central European Time (UTC+1), please consider this especially, when contributing your own data."
        )
    ),
    ui.head_content(
        ui.tags.script("""
            Shiny.addCustomMessageHandler("download_file", function(message) {
                var link = document.createElement('a');
                link.href = 'data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,' + message.data;
                link.download = message.filename;
                document.body.appendChild(link);
                link.click();
                document.body.removeChild(link);
            });
        """),
        ui.tags.link(rel="icon", type="image/png", href="/www/WPP_icon2.png")
    ),
    id="navbar_selected",
    title="Wind Power Forecast"
)

# server function
def server(input, output, session):

    ##### Page 1 #####

    # Define reactive values for storing additional information
    project_name = reactive.Value(None)
    operator = reactive.Value(None)
    owner = reactive.Value(None)

    is_programmatic_change = reactive.Value(False)

    @reactive.effect
    @reactive.event(input.entire_forecast)
    def entire_forecast_function():
        # Setzen des Flags: Änderungen sind programmatisch
        is_programmatic_change.set(True)

        id = input.entire_forecast()['id']

        index = list(ids).index(id)

        # Speichern der zusätzlichen Informationen in den reaktiven Werten
        project_name.set(project_names[index])
        operator.set(operators[index])
        owner.set(owners[index])

        # Parameter extrahieren
        lat = lats_plants[index]
        lon = lons_plants[index]
        turbine_type = turbine_types[index]
        hub_height = hub_heights[index]
        capacity = capacities[index]
        commissioning_date = commissioning_dates[index]
        commissioning_date_year, commissioning_date_month = commissioning_date.split("/")
        # commissioning_date_year = int(commissioning_date_year)
        # commissioning_date_month = int(commissioning_date_month)

        # Update der Eingabefelder mit neuen Werten vor Wechseln des Tabs
        ui.update_numeric("lat", value=lat)
        ui.update_numeric("lon", value=lon)
        ui.update_select("turbine_type", selected=turbine_type)
        ui.update_slider("hub_height", value=hub_height)
        ui.update_slider("commissioning_date_year", value=commissioning_date_year)
        ui.update_slider("commissioning_date_month", value=commissioning_date_month)
        ui.update_slider("capacity", value=capacity)

        # Wechsel zu "Customise WPP" Tab
        ui.update_navs("navbar_selected", selected="customise_WPP")

    @output
    @render_widget
    def map():
        m = Map(
            center=[(lat_min + lat_max) / 2, (lon_min + lon_max) / 2],
            zoom=5,
            layout=Layout(width='100%', height='95vh'),
            scroll_wheel_zoom=True
        )

        markers = []
        for name, capacity, operator, id, lat, lon in zip(project_names, capacities, operators, ids, lats_plants, lons_plants):
            
            popup_content = HTML(
                f"<strong>Project Name:</strong> {name}<br>"
                f"<strong>Capacity:</strong> {capacity} kW<br>"
                f"<strong>Operator:</strong> {operator}<br>"
                f"<strong>Wind speed forecast:</strong> select forecast step<br>"
                f"<strong>Production forecast:</strong> select forecast step<br>"
                f"<button onclick=\"Shiny.setInputValue(\'entire_forecast\', {{id: {id}}})\">Entire Forecast</button>"
            )

            marker = Marker(
                location=(lat, lon),
                popup=popup_content,
                rise_offset=True,
                draggable=False
            )
            markers.append(marker)

        # Cluster erstellen und zur Karte hinzufügen
        marker_cluster = MarkerCluster(markers=markers)
        m.add(marker_cluster)

        # Slider for time steps
        play = Play(min=0, max=total_hours, step=step_size_hours, value=0, interval=500, description='Time Step')
        slider = SelectionSlider(options=valid_times_interpol, value=valid_times_interpol[0], description='Time')
        jslink((play, 'value'), (slider, 'index'))
        slider_box = VBox([play, slider])
        m.add(WidgetControl(widget=slider_box, position='topright'))

        # Map update function based on slider
        def update_map(change):
            time_step = slider.value
            step_index = int((time_step - start_time) / step_size)
            spatial_interpolator = interp2d(lons, lats, total_selection_interpol[step_index], kind='cubic')
            wind_speeds_at_points = np.array([spatial_interpolator(lon, lat)[0] for lon, lat in zip(lons_plants, lats_plants)])
            #wind_speeds_at_points = np.nan_to_num(wind_speeds_at_points, nan=0.0) # later: remove WPPs with nan values for wind, i. e. at the edge of Europe

            all_input_features = np.hstack([
                turbine_types_onehot,
                hub_heights,
                capacities,
                ages_months,
                wind_speeds_at_points
            ])

            input_tensor = torch.tensor(all_input_features, dtype=torch.float32)

            with torch.no_grad():
                predictions = model(input_tensor).numpy().flatten()
            
            # Update marker pop-ups with production values
            for marker, name, capacity, operator, wind_speed, prediction, id in zip(marker_cluster.markers, project_names, capacities, operators, wind_speeds_at_points, predictions, ids):
                marker.popup.value = \
                    f"<strong>Project Name:</strong> {name}<br>"\
                    f"<strong>Capacity:</strong> {capacity} kW<br>"\
                    f"<strong>Operator:</strong> {operator}<br>"\
                    f"<strong>Wind speed forecast:</strong> {wind_speed:.2f} m/s<br>"\
                    f"<strong>Production forecast:</strong> {prediction:.2f} kW<br>"\
                    f"<button onclick=\"Shiny.setInputValue(\'entire_forecast\', {{id: {id}}})\">Entire Forecast</button>"
                
            # Prepare heatmap data for "operating" wind plants
            heatmap_data = [(lat, lon, prod) for lat, lon, prod in zip(lats_plants, lons_plants, predictions)]

            # Remove existing heatmap layer if any and add new one
            for layer in m.layers:
                if isinstance(layer, Heatmap):
                    m.remove(layer)
            heatmap = Heatmap(locations=heatmap_data, radius=5, blur=5, max_zoom=10)
            m.add(heatmap)
        
        global initial
        # Initialise the map with the first time step
        if initial == 0:
            update_map(None)
            initial = 1
        
        # Observe slider value changes and update map
        slider.observe(update_map, names='value')

        # Add FullScreenControl to map
        m.add(FullScreenControl())

        return m
    

    ##### Page 2 #####

    forecast_data = reactive.Value({"wind_speeds": None, "productions": None})
    time_series = reactive.Value(None)
    button_status = reactive.Value("download")

    # Function to handle file upload
    @reactive.Effect
    @reactive.event(input.upload_file)
    def handle_file_upload():
        if input.upload_file() is not None:
            try:
                file = input.upload_file()[0]['datapath']
                # Attempt to read Excel file with specified columns
                time_series_data = pd.read_excel(file)
                time_series.set(time_series_data) # calls output_graph function
                ui.update_action_button("action_button", label="Contribute Data")
                button_status.set('contribute')
            except Exception as e:
                # Send feedback to UI if file format is incorrect
                ui.notification_show("Wrong file format, please download the example file for orientation.", duration=None)
    
    # Generate example file for download
    @reactive.Effect
    @reactive.event(input.download_example)
    async def generate_example_file():
        # Create example workbook
        wb = Workbook()
        ws = wb.active
        ws.title = "Example Time Series"
        
        # Add header and example data
        ws.append(["Date", "Production (kW)"])
        for date, production in zip(example_dates, example_production):
            ws.append([date, production])
        
        # Save workbook to buffer
        buffer = io.BytesIO()
        wb.save(buffer)
        buffer.seek(0)
        
        # Encode and send for download
        file_data_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        await session.send_custom_message("download_file", {
            "data": file_data_base64,
            "filename": "example_time_series.xlsx"
        })

    # Observing slider changes to revert to forecast view
    @reactive.Effect
    @reactive.event(input.lat, input.lon, input.turbine_type, input.hub_height, input.commissioning_date_year, input.commissioning_date_month, input.capacity)
    def observe_slider_changes():
        # Überspringen, wenn Änderungen programmatisch sind
        if is_programmatic_change.get():
            return
        
        # reset reactive variables
        project_name.set(None)
        operator.set(None)
        owner.set(None)

        # display regular elements
        time_series.set(None) # calls output_graph function
        ui.update_action_button("action_button", label="Download Forecast")
        button_status.set('download')

        # Zurücksetzen des Flags
        is_programmatic_change.set(False)

    # Capture user input and display configuration summary
    @output
    @render.text
    def output_summary():
        # Capture inputs
        lat_plant = input.lat()
        lon_plant = input.lon()
        turbine_type = input.turbine_type()
        hub_height = input.hub_height()
        commissioning_date_year = input.commissioning_date_year()
        commissioning_date_month = input.commissioning_date_month()
        capacity = input.capacity()

        project_name_value = project_name.get()

        # Return configuration summary text
        summary_html = (
            f"<b>Turbine Configuration</b><br><br>"
            f"<b>Project Name:</b> {project_name_value}<br>"
            f"<b>Operator:</b> {operator.get()}<br>"
            f"<b>Owner:</b> {owner.get()}<br>"
            f"<b>Location:</b> ({lat_plant}, {lon_plant})<br>"
            f"<b>Type:</b> {turbine_type}<br>"
            f"<b>Hub Height:</b> {hub_height} m<br>"
            f"<b>Commissioning Date:</b> {commissioning_date_year}/{commissioning_date_month}<br>"
            f"<b>Capacity:</b> {capacity} kW<br>"
        )
        return ui.HTML(summary_html)

    # Plot forecasted production over time
    @output
    @render.plot
    def output_graph():
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.set_xlabel("Date")
        ax.set_ylabel("Production (kW)")

        # Add horizontal dashed line for capacity
        capacity = input.capacity()
        ax.axhline(y=capacity, color='gray', linestyle='--', label=f"Capacity ({capacity} kW)")

        time_series_data = time_series.get()

        if time_series_data is not None and not time_series_data.empty: # check if user has uploaded time series to plot it
            ax.plot(pd.to_datetime(time_series_data.iloc[:, 0], errors='coerce'), time_series_data.iloc[:, 1], label="Historical Production (kW)")
            ax.set_title("Historical Wind Turbine Production")
        else: # Retrieve inputs
            lat_plant = input.lat()
            lon_plant = input.lon()
            turbine_type = input.turbine_type()
            hub_height = input.hub_height()
            commissioning_date_year = input.commissioning_date_year()
            commissioning_date_month = input.commissioning_date_month()
            ref_date = pd.Timestamp("2024-12-01")
            age_months = (ref_date.year - commissioning_date_year) * 12 + (ref_date.month - commissioning_date_month)
            capacity = input.capacity()

            # Calculate forecasted productions
            wind_speeds_at_point = []
            num_steps = len(valid_times_interpol)
            for step_index in range(num_steps):
                spatial_interpolator = interp2d(lons, lats, total_selection_interpol[step_index], kind='cubic')
                wind_speeds_at_point.append(spatial_interpolator(lon_plant, lat_plant)[0])

            turbine_type_repeated = np.tile(encoder.transform(turbine_type), (num_steps, 1))
            hub_height_repeated = np.full((num_steps, 1), hub_height)
            capacity_repeated = np.full((num_steps, 1), capacity)
            age_repeated = np.full((num_steps, 1), age_months)
            wind_speed_column = wind_speeds_at_point.reshape(-1, 1)

            all_input_features = np.hstack([
                turbine_type_repeated,
                hub_height_repeated,
                capacity_repeated,
                age_repeated,
                wind_speed_column
            ])

            input_tensor = torch.tensor(all_input_features, dtype=torch.float32)

            with torch.no_grad():
                predictions = model(input_tensor).numpy().flatten()

            # Store the forecast data in the reactive `forecast_data`
            forecast_data.set({"wind_speeds": wind_speeds_at_point, "productions": predictions})
            ax.plot(valid_times_interpol, predictions, label="Predicted Production (kW)")
            ax.set_title("Forecasted Wind Turbine Production")

        ax.legend()
        ax.grid(True)
        plt.tight_layout()
        return fig  # Returning the figure will embed it in the app

    def download_forecast_data():

        data = forecast_data.get()
        wind_speeds, productions = data["wind_speeds"], data["productions"]

        lat_plant = input.lat()
        lon_plant = input.lon()
        turbine_type = input.turbine_type()
        hub_height = input.hub_height()
        commissioning_date_year = input.commissioning_date_year()
        commissioning_date_month = input.commissioning_date_month()
        capacity = input.capacity()

        # Retrieve the actual values of reactive objects
        project_name_value = project_name.get()
        operator_value = operator.get()
        owner_value = owner.get()

        # Create a new workbook and add data
        wb = Workbook()
        
        # Sheet1: Turbine Specifications
        ws_specs = wb.active
        ws_specs.title = "Turbine Specifications"

        # Define the specs_data list with required values only
        specs_data = [
            ["Specification", "Value"],
            ["Project Name", project_name_value],
            ["Operator", operator_value],
            ["Owner", owner_value],
            ["Location", f"({lat_plant}, {lon_plant})"],
            ["Type", turbine_type],
            ["Hub Height (m)", hub_height],
            ["Commissioning Date", str(commissioning_date_year) + "/" + str(commissioning_date_month)],
            ["Capacity (kW)", capacity]
        ]

        # Populate the workbook
        for row in specs_data:
            ws_specs.append(row)

        # Sheet2: Production Forecast
        ws_forecast = wb.create_sheet("Production Forecast")

        # Add headers for date, wind speed, and production
        ws_forecast.append(["Date", "Wind Speed (m/s)", "Production (kW)"])

        # Add production data per time step
        for time, wind_speed, production in zip(valid_times_interpol, wind_speeds, productions):
            time_as_datetime = pd.to_datetime(time).to_pydatetime()
            ws_forecast.append([time_as_datetime, wind_speed, production])  # Ensure wind speed and production values are correctly added

        # Save in buffer and return
        buffer = io.BytesIO()
        wb.save(buffer)
        buffer.seek(0)
        return buffer.getvalue()

    # Function to trigger download on button click
    @reactive.Effect
    @reactive.event(input.action_button)
    async def action():
        if button_status.get() == "download":
        
            # Retrieve forecast data as bytes
            file_data = download_forecast_data()
            
            # Encode file in Base64
            file_data_base64 = base64.b64encode(file_data).decode('utf-8')
            
            # Send file as Base64 to client for download
            await session.send_custom_message("download_file", {
                "data": file_data_base64,
                "filename": "forecasted_production.xlsx"
            })
        
        else: # button_status.get() == "contribute"
            ui.notification_show(
                "Thank you for uploading your time series. It will be checked for plausibility and possibly used to improve the model at the next training.",
                duration=None
            )
            # save time_series somewhere until training
            time_series.set(None)

# Define absolute path to `www` folder
path_www = os.path.join(os.getcwd(), "www")

# Start the app with the `www` directory as static content
app = App(app_ui, server, static_assets={"/www": path_www})

if __name__ == "__main__":
    # Check if the variable `RENDER` is set to detect if the app is running on Render
    if os.getenv("RENDER") or os.getenv("WEBSITE_HOSTNAME"):  # for Render or Azure Server
        host = "0.0.0.0"  # For Render or other external deployments
    else:
        host = "127.0.0.1"  # For local development (localhost)

    app.run(host=host, port=8000)  # port binding: set the server port to 8000, because this is what Azure expects


Ignoring index file 'data/weather_forecast/data_europe.grib2.5b7b6.idx' older than GRIB file


Data is current and will be used.


C:\Users\alexa\AppData\Local\Temp\ipykernel_24108\1234309728.py:179: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load(model_path)


Modell erfolgreich geladen mit input_size=50


RuntimeError: asyncio.run() cannot be called from a running event loop

In [13]:
max_year

2023